<a href="https://colab.research.google.com/github/Matheusbcy/-Data-Science-IA-/blob/main/Compreens%C3%A3o_e_Transcri%C3%A7%C3%A3o_de_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto - Trancriçao e compreensão de vídeos

## Instalação e Configuração


In [ ]:
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai langchain-groq

### Instalação de bibliotecas para baixar transcrição

#### YouTube Transcript API

In [ ]:
!pip install youtube-transcript-api

#### pytube

In [ ]:
!pip install pytube

## Importações


In [48]:
import os
import io
import getpass
import requests

from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from langchain_community.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown

### Carregando a transcrição

In [29]:
# ID do vídeo do YouTube. Verifique pela URL, é o que vem logo após /watch?v=
id_video = "46Gib0GD4Gw"

# Busca a transcrição, tentando em português e inglês
result = YouTubeTranscriptApi().fetch(id_video, languages=["pt", "pt-BR", "en"])

# TextFormatter para extrair apenas o texto da transcrição, sem timestamps, e unificamos tudo em um único bloco contínuo
formatter = TextFormatter()
transcricao = formatter.format_transcript(result)

# opcional: Substitui quebras de linha por espaços para manter tudo em uma só linha (Se não quiser isso, remova essa linha abaixo)
transcricao = transcricao.replace('\n', ' ')

## Obter informações do vídeo

### Titulo do Video

In [30]:
def get_video_title(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text)

  link = soup.find_all(name="title")[0]
  title = str(link)
  title = title.replace("<title>","")
  title = title.replace("</title>","")

  return title

In [34]:
video_url = "https://www.youtube.com/watch?v=46Gib0GD4Gw&ab_channel=Bucky"

video_title = get_video_title(video_url)

In [35]:
infos_video = f"""
Título: {video_title}

Transcrição: {transcricao}
"""

### Salvando transcrição em um arquivo

In [36]:
with io.open("transcricao.txt", "w", encoding= "utf-8") as f:
    f.write(infos_video)

## Carregamento do modelo

In [10]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [11]:
def model_groq(model = "gemma2-9b-it", temperature = 0.7):
  llm = ChatGroq(model = model, max_tokens=1024, timeout=None, max_retries=2, temperature = temperature)

  return llm

In [12]:
llm = model_groq()

## Criação do prompt template

In [ ]:
system_prompt = "Você é um assistente virtual prestativo e deve responder a consulta com base na transcrição de um video, que sera fornecido abaixo."

inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"

user_prompt = "{}".format(inputs)

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

chain = prompt_template | llm | StrOutputParser()

## Criação da chain

## Geração da resposta

In [ ]:
res = chain.invoke({"transcrição": transcricao, "consulta": "resuma"})
print(res)

In [ ]:
res = chain.invoke({"transcrição": transcricao, "consulta": "sumarize de forma clara de entender em pt-br"})
print(res)

In [ ]:
res = chain.invoke({"transcrição": transcricao, "consulta": "explique em 1 frase sobre oque fala esse video"})
print(res)

In [ ]:
res = chain.invoke({"transcrição": transcricao, "consulta": "liste os temas desse video"})
display(Markdown(res))

In [50]:
res = chain.invoke({"transcrição": transcricao, "consulta": "liste os temas desse video, responsa em portugues brasil, utilize markdown no retorno da resposta"})
display(Markdown(res))

## Temas do vídeo:

* **Progresso no jogo "Halfsword":** O criador do vídeo finalmente consegue vencer o jogo após enfrentar problemas de crashes e dificuldades com o gameplay.
* **Dificuldades e estratégias:** O vídeo mostra os desafios enfrentados pelo jogador, como encontrar armas específicas (espada longa, spear), lidar com inimigos fortes (Golden Knight) e melhorar o equipamento (armadura).
* **Humor e interação com o público:** O vídeo é recheado de humor, com comentários sarcásticos e ironia sobre os desafios e personagens do jogo. O criador se dirige ao público, perguntando sobre futuros desafios e interagindo com o que ele considera "bromance" com os espectadores. 
* **Melhorias no desempenho:** O criador menciona a compra de uma nova placa de vídeo para solucionar o problema de crashes, demonstrando a importância do hardware para o jogo.
* **Desbloqueamento de conteúdo:** O vídeo celebra o desbloqueio da última fase e a conquista de novas opções de personagens e equipamentos. 


